In [2]:
from dbhandler import DBHandler
from typing import Optional
import datetime

from langchain.prompts.chat import (ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate,)
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts.prompt import PromptTemplate

def filter_journal_entries(student_name, start_date, end_date):

    db_handler = DBHandler()

    # Initialize empty lists to store all journal entries and metadatas
    metadatas = []
    journal_entries = []
    
    # Initialize student attributes
    student = db_handler.get_student(student_name)
    student_id = student['_id']  # Name of the student
    class_ = student['class'] # Student's class 

    journals = db_handler.get_journal_entries(student_id)

    # Iterate over all journal entries for the student
    for journal in journals:
        journal_id = journal['_id'] # ID of the journal entry
        journal_title = journal['title']
        journal_content = journal['content']
        date = journal['date']

        # Check if the journal entry falls within the date range
        if start_date <= date <= end_date:
            # Append to our journal entries list
            metadatas.append({
                'student_id': str(student_id),
                'student_name': student_name,
                'class': class_,
                'source': str(journal_id),
                'journal_title': journal_title,
                'journal_date': str(date)}
            )

            # Append content only
            journal_entries.append(journal_content)

    return journal_entries, metadatas

def convert_journal_entries_to_vectors(journal_entries, metadatas):
    # Instantiate OpenAIEmbeddings
    openai_embeddings = OpenAIEmbeddings(model='gpt-3.5-turbo')
    vector_store = Chroma.from_texts(journal_entries, 
                                 embedding=openai_embeddings, 
                                 metadatas=metadatas)
    return vector_store

def process_query(question, vector_store, student_name):

    system_template = f"The student's name is {student_name}" + """Use the following pieces of context to answer the user's question.\
    No matter what the question is, you should always answer it in the context of the journal entries provided.\
    If possible, point the user to the journal entry where you obtained the information from. \
    Even if the question does not end in a question mark, you should still answer it as if it were a question.\
    If the question is not related to the student in question, reply "Please ask me questions about the student in question".\
    ----------------
    {summaries}"""

    messages = [
        SystemMessagePromptTemplate.from_template(system_template),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]

    prompt = ChatPromptTemplate.from_messages(messages)

    language_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) 

    chain_type_kwargs = {"prompt": prompt}

    chain = RetrievalQAWithSourcesChain.from_chain_type(
        llm=language_model,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_type="similarity"),
        return_source_documents=True,
        chain_type_kwargs=chain_type_kwargs
    )

    result = chain(question)
    return result

In [3]:
def main(student_name, start_date, end_date, question):
    journal_entries, metadatas = filter_journal_entries(student_name, start_date, end_date)
    vector_store = convert_journal_entries_to_vectors(journal_entries, metadatas)
    return process_query(question, vector_store, student_name)

In [4]:
student_name = 'Li Wei Tan'
start_date_str = '2023-02-15'
start_date_obj = datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
end_date_str = '2023-04-15'
end_date_obj = datetime.datetime.strptime(end_date_str, '%Y-%m-%d')

In [5]:
main(student_name, start_date_obj, end_date_obj, "Are there any issues about Li Wei that I, as a teacher, should be concerned about?")

Using embedded DuckDB without persistence: data will be transient


{'question': 'Are there any issues about Li Wei that I, as a teacher, should be concerned about?',
 'answer': 'There is no information in the provided journal entries that suggests any issues or concerns about Li Wei. In fact, Li Wei seems to prioritize self-care, social connections, and personal growth, which are all positive qualities for a student. Source: 645ca721f91d80b63f70aa61, 645ca721f91d80b63f70aa5f, 645ca722f91d80b63f70aa64, 645ca721f91d80b63f70aa60.',
 'sources': '',
 'source_documents': [Document(page_content="Today was a bit of a difficult day. I had a test in math class that I was really nervous about, and I didn't do as well as I had hoped. I was also feeling a bit overwhelmed with all the work I have to do for school. But I'm trying to stay positive and focus on the things I can do to make things better. I'm going to take some time to relax and do something I enjoy, and I'm going to try to get a good night's sleep so I can be ready for tomorrow.", metadata={'student_id

In [6]:
main(student_name, start_date_obj, end_date_obj, "How would you describe Li Wei's sentiments or emotional health in this period?")

Using embedded DuckDB without persistence: data will be transient


{'question': "How would you describe Li Wei's sentiments or emotional health in this period?",
 'answer': 'Li Wei Tan seems to be someone who values creativity, kindness, and self-care. They try to practice self-care every day and make sure to take time to do things they enjoy. However, they also have difficult days, like the one described in the second journal entry, where they feel overwhelmed and nervous. Despite this, they try to stay positive and focus on the things they can do to make things better. Overall, Li Wei Tan seems to prioritize their emotional health and wellbeing.',
 'sources': '',
 'source_documents': [Document(page_content='I value creativity, kindness, and self-care. I believe that creativity is an important part of life, and that it should be nurtured and encouraged. I also believe that kindness is essential for a happy and healthy life, and that it should be practiced and shared. Lastly, I believe that self-care is important for physical and mental wellbeing, and